In [102]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [103]:
merged_df = pd.read_csv("../mega_df.csv")
merged_df = merged_df.rename(columns={'continent':'Continent', 'country_code':'Country Code', 'country':'Country', 'consumption_per_capita_(L)':'Alcohol Consumption Per Capita (Liters)', 'climate':'Climate', 'main_export':'Main Export', 'main_industry':'Main Industry', 'region':'Region', 'happy_score':'Happiness Score', 'life_expectancy':'Life Expectancy', 'gdp_per_capita':'GDP per capita', 'mental_health_and_addiction (%)':'Mental Health Disorders (%)'})
merged_df

,Unnamed: 0,Continent,Country Code,Country,Alcohol Consumption Per Capita (Liters),Climate,Main Export,Main Industry,Region,Happiness Score,Life Expectancy,GDP per capita,code,Mental Health Disorders (%)
0,0,Asia,AFG,Afghanistan,0.2,arid to semiarid; cold winters and hot summers,Fruit and nuts,small-scale production of bricks,South Asia,2.5669,52.590000,7.462861,AFG,17.553463
1,1,Europe,ALB,Albania,7.5,"mild temperate; cool, cloudy, wet winters; hot...",Chromium and chrome products,perfumes and cosmetic products,Central and Eastern Europe,4.8827,68.708138,9.417931,ALB,10.987610
2,4942,Africa,DZA,Algeria,0.9,"arid to semiarid; mild, wet winters with hot, ...",Oil,petroleum,Middle East and North Africa,5.0051,65.905174,9.537965,DZA,14.700388
3,4943,Europe,AND,Andorra,11.3,"temperate; snowy, cold winters and warm, dry s...",Tobacco products,tourism,NaN,NaN,NaN,NaN,AND,14.726869
4,4944,Africa,AGO,Angola,6.4,semiarid in south and along coast to Luanda; n...,Oil,petroleum,NaN,NaN,NaN,NaN,AGO,12.498281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,5128,Asia,UZB,Uzbekistan,2.7,"mostly mid-latitude desert, long, hot summers,...",Cotton,textiles,Commonwealth of Independent States,6.2576,65.107574,8.740372,UZB,10.897074
182,5129,South America,VEN,Venezuela,5.6,"tropical; hot, humid; more moderate in highlands",Petroleum,petroleum,Latin America and Caribbean,5.0532,66.505341,8.977794,VEN,11.014506
183,5130,Oceania,WSM,Samoa,2.5,"tropical; rainy season (November to April), dr...",Coconut oil and cream,food processing,NaN,NaN,NaN,NaN,WSM,11.152716
184,5131,Asia,YEM,Yemen,0.1,mostly desert; hot and humid along west coast;...,Crude oil,crude oil production and petroleum refining,Middle East and North Africa,3.5274,56.727283,7.759683,YEM,15.819471


In [104]:
updated_df=merged_df.drop(['Unnamed: 0', 'code'], axis=1)
updated_df.head()

,Continent,Country Code,Country,Alcohol Consumption Per Capita (Liters),Climate,Main Export,Main Industry,Region,Happiness Score,Life Expectancy,GDP per capita,Mental Health Disorders (%)
0,Asia,AFG,Afghanistan,0.2,arid to semiarid; cold winters and hot summers,Fruit and nuts,small-scale production of bricks,South Asia,2.5669,52.590000,7.462861,17.553463
1,Europe,ALB,Albania,7.5,"mild temperate; cool, cloudy, wet winters; hot...",Chromium and chrome products,perfumes and cosmetic products,Central and Eastern Europe,4.8827,68.708138,9.417931,10.987610
2,Africa,DZA,Algeria,0.9,"arid to semiarid; mild, wet winters with hot, ...",Oil,petroleum,Middle East and North Africa,5.0051,65.905174,9.537965,14.700388
3,Europe,AND,Andorra,11.3,"temperate; snowy, cold winters and warm, dry s...",Tobacco products,tourism,NaN,NaN,NaN,NaN,14.726869
4,Africa,AGO,Angola,6.4,semiarid in south and along coast to Luanda; n...,Oil,petroleum,NaN,NaN,NaN,NaN,12.498281


In [105]:
updated_df["Main Export"].value_counts().head(20)

Oil                                  11
Machinery and transport equipment     8
Petroleum                             8
Coffee                                7
Crude oil                             5
Cotton                                5
Manufactured goods                    5
Diamonds                              4
Gold                                  4
Machinery and equipment               3
Fish and fish products                3
Sugar                                 3
Cocoa                                 3
Bananas                               3
Bauxite                               3
Fish                                  3
Machinery                             3
Tobacco                               2
Copra                                 2
Agricultural products                 2
Name: Main Export, dtype: int64

In [109]:
# updated_df["Main Export"] = np.where(updated_df['group'].isin(['B','G','H']), 'NC',
# updated_df$group=as.character(updated_df$group)
# updated_df$group[c(3:4,13:16)]='NC'
# updated_df$group[c(1:2)]='PC'
# updated_df$group[c(5:12)]='NON'
# updated_df["Main Export"] = np.where(updated_df['Main Export'].isin(['B','G','H']), 'Petroleum',
#               np.where(updated_df['Main Export'] == 'A', 'PC', 'NON'))
updated_df["Main Export"] = np.where(updated_df["Main Export"].isin(["petroleum"]),"Petroleum")
updated_df.sample(30)

ValueError: either both or neither of x and y should be given

In [107]:
updated_df = updated_df.replace(
    {"Crude oil": "Petroleum", "Oil": "Petroleum", "Textile products":"Textiles and clothing", "Fish":"Agricultural products", "Aluminium":"Semi-precious metals", "Vehicles":"Machinery", "Bauxite and alumina":"Semi-precious metals", "Fuels and fuel products":"Petroleum", "Soyabeans":"Agricultural products", "Garments":"Textiles and clothing", "Clothing":"Textiles and clothing", "Computers and electrical machinery":"Machinery", "Machinery and electrical equipment":"Machinery", "Peetroleum":"Petroleum", "Copra":"Semi-precious metals", "Clothing and textiles":"Textiles and clothing", "Petroleum and petroleum products":"Petroleum", "Fish and fish products":"Agricultural products", "Machinery and transport equipment":"Machinery", "Machinery and equipment":"Machinery", "Machinery and electronics":"Machinery", "Bananas":"Agricultural products", "Sugar":"Agricultural products", "Fruit":"Agricultural products", "Coffee":"Agricultural products", "Food including rice":"Agricultural products", "Tobacco products": "Manufactured goods", "Ores and metals":"Semi-precious metals", "Foodstuffs":"Agricultural products", "Tobacco":"Agricultural products", "Vanilla": "Agricultural products", "Cocoa":"Agricultural products", "Bauxite":"Semi-precious metals", "Nickel":"Semi-precious metals", "Copper":"Semi-precious metals", "Tea":"Agricultural products", "Diamonds":"Precious stones and metals", "Gold":"Precious stones and metals"})
updated_df

,Continent,Country Code,Country,Alcohol Consumption Per Capita (Liters),Climate,Main Export,Main Industry,Region,Happiness Score,Life Expectancy,GDP per capita,Mental Health Disorders (%)
0,Asia,AFG,Afghanistan,0.2,arid to semiarid; cold winters and hot summers,Fruit and nuts,small-scale production of bricks,South Asia,2.5669,52.590000,7.462861,17.553463
1,Europe,ALB,Albania,7.5,"mild temperate; cool, cloudy, wet winters; hot...",Chromium and chrome products,perfumes and cosmetic products,Central and Eastern Europe,4.8827,68.708138,9.417931,10.987610
2,Africa,DZA,Algeria,0.9,"arid to semiarid; mild, wet winters with hot, ...",Petroleum,petroleum,Middle East and North Africa,5.0051,65.905174,9.537965,14.700388
3,Europe,AND,Andorra,11.3,"temperate; snowy, cold winters and warm, dry s...",Manufactured goods,tourism,NaN,NaN,NaN,NaN,14.726869
4,Africa,AGO,Angola,6.4,semiarid in south and along coast to Luanda; n...,Petroleum,petroleum,NaN,NaN,NaN,NaN,12.498281
...,...,...,...,...,...,...,...,...,...,...,...,...
181,Asia,UZB,Uzbekistan,2.7,"mostly mid-latitude desert, long, hot summers,...",Cotton,textiles,Commonwealth of Independent States,6.2576,65.107574,8.740372,10.897074
182,South America,VEN,Venezuela,5.6,"tropical; hot, humid; more moderate in highlands",Petroleum,petroleum,Latin America and Caribbean,5.0532,66.505341,8.977794,11.014506
183,Oceania,WSM,Samoa,2.5,"tropical; rainy season (November to April), dr...",Coconut oil and cream,food processing,NaN,NaN,NaN,NaN,11.152716
184,Asia,YEM,Yemen,0.1,mostly desert; hot and humid along west coast;...,Petroleum,crude oil production and petroleum refining,Middle East and North Africa,3.5274,56.727283,7.759683,15.819471


In [108]:
updated_df["Main Export"].value_counts().head(30)

Agricultural products                 35
Petroleum                             27
Machinery                             19
Semi-precious metals                  12
Textiles and clothing                  9
Precious stones and metals             8
Manufactured goods                     6
Cotton                                 5
Phosphates                             2
Water                                  1
Steel products                         1
Military equipment                     1
Re-exports                             1
Electronic equipment                   1
Processed and unprocessed diamonds     1
Meat                                   1
Uranium                                1
Light manufactures                     1
Timber                                 1
Chromium and chrome products           1
Livestock                              1
Sugar and molasses                     1
Computer software                      1
Foodstuffs and tobacco                 1
Shoes           

In [112]:
top_export_data = updated_df.loc[(updated_df["Main Export"]=="Petroleum") | (updated_df["Main Export"]=="Machinery") | (updated_df["Main Export"]=="Semi-precious metals") | (updated_df["Main Export"]=="Textiles and clothing") | (updated_df["Main Export"]=="Precious stones and metals") | (updated_df["Main Export"]=="Manufactured goods")]
top_export_data

,Continent,Country Code,Country,Alcohol Consumption Per Capita (Liters),Climate,Main Export,Main Industry,Region,Happiness Score,Life Expectancy,GDP per capita,Mental Health Disorders (%)
2,Africa,DZA,Algeria,0.9,"arid to semiarid; mild, wet winters with hot, ...",Petroleum,petroleum,Middle East and North Africa,5.0051,65.905174,9.537965,14.700388
3,Europe,AND,Andorra,11.3,"temperate; snowy, cold winters and warm, dry s...",Manufactured goods,tourism,NaN,NaN,NaN,NaN,14.726869
4,Africa,AGO,Angola,6.4,semiarid in south and along coast to Luanda; n...,Petroleum,petroleum,NaN,NaN,NaN,NaN,12.498281
5,North America,ATG,Antigua and Barbuda,7.0,tropical maritime; little seasonal temperature...,Textiles and clothing,tourism,NaN,NaN,NaN,NaN,13.159940
6,Europe,AZE,Azerbaijan,0.8,"dry, semiarid steppe",Petroleum,petroleum and natural gas,Commonwealth of Independent States,5.1648,65.508400,9.687727,10.495815
...,...,...,...,...,...,...,...,...,...,...,...,...
177,Africa,TZA,Tanzania,9.4,varies from tropical along coast to temperate ...,Precious stones and metals,agricultural processing (sugar,Sub-Saharan Africa,3.4762,57.496075,7.967665,12.637257
178,North America,USA,United States,9.8,"mostly temperate, but tropical in Hawaii and F...",Machinery,highly diversified,North America and ANZ,6.9396,68.299500,10.925769,17.080892
182,South America,VEN,Venezuela,5.6,"tropical; hot, humid; more moderate in highlands",Petroleum,petroleum,Latin America and Caribbean,5.0532,66.505341,8.977794,11.014506
184,Asia,YEM,Yemen,0.1,mostly desert; hot and humid along west coast;...,Petroleum,crude oil production and petroleum refining,Middle East and North Africa,3.5274,56.727283,7.759683,15.819471


In [35]:
updated_df["Main Industry"].value_counts().head(15)

tourism                      17
petroleum                    12
food processing              10
textiles                      9
sugar                         5
mining                        4
fishing                       4
food and beverages            3
construction                  3
oil                           3
petroleum and natural gas     2
phosphate mining              2
aluminum                      2
cement                        2
electronics                   2
Name: Main Industry, dtype: int64